**Table of contents**<a id='toc0_'></a>    
- [The project](#toc1_)    
  - [1. Descriptive Analysis](#toc1_1_)    
    - [1.1 Missing values](#toc1_1_1_)    
    - [1.2 Outliers](#toc1_1_2_)    
    - [1.3 Correlation analysis](#toc1_1_3_)    
    - [1.4 Target variable](#toc1_1_4_)    
  - [2. Data cleaning for train and test sets](#toc1_2_)    
    - [2.1 Split datasets into train and test sets](#toc1_2_1_)    
    - [2.2 Handling missing values](#toc1_2_2_)    
      - [2.2.1 Imputation for train set](#toc1_2_2_1_)    
      - [2.2.2 Imputation for test set](#toc1_2_2_2_)    
  - [3. Data processing approaches](#toc1_3_)    
    - [3.1 Approach 1: Baseline Model (Full Data)](#toc1_3_1_)    
    - [3.2 Approach 2: Correlation-Based Feature Selection & Outlier Removal](#toc1_3_2_)    
      - [3.2.1 Correlation-Based Feature Selection for train and test sets separately](#toc1_3_2_1_)    
      - [3.2.2 Outlier Removal for train set](#toc1_3_2_2_)    
      - [3.2.3 Statistical analysis](#toc1_3_2_3_)    
        - [1) The proportion of data points per ERs](#toc1_3_2_3_1_)    
        - [2) TOP-10 significant features](#toc1_3_2_3_2_)    
    - [3.3 Approach 3: Statistical Distribution-Based Feature Selection ](#toc1_3_3_)    
      - [3.3.1 Statistical Distribution-Based Feature Selection ](#toc1_3_3_1_)    
      - [3.3.2 Statistical analysis](#toc1_3_3_2_)    
        - [1) The proportion of data points per ERs](#toc1_3_3_2_1_)    
        - [2) TOP-10 statistically significant features](#toc1_3_3_2_2_)    
    - [3.4 Approach 4: Non-Linear Dimensionality Reduction & Scaling](#toc1_3_4_)    
      - [3.4.1 Scaling: Robust Scaler for train and test sets separately](#toc1_3_4_1_)    
      - [3.4.2 Dimensionality reduction: UMAP for train and test sets separately](#toc1_3_4_2_)    
      - [3.3.2 Statistical analysis](#toc1_3_4_3_)    
        - [1) TOP-10 significant components](#toc1_3_4_3_1_)    
  - [4. Conclusions](#toc1_4_)    
    - [**1. Baseline (Approach 1)**](#toc1_4_1_)    
    - [**2. Correlation-Based Selection & Outlier Removal (Approach 2)**](#toc1_4_2_)    
    - [**3. Statistical Distribution-Based Selection (Approach 3)**](#toc1_4_3_)    
    - [**4. Non-Linear Dimensionality Reduction + Robust Scaling (Approach 4)**](#toc1_4_4_)    
    - [**General Observations and Recommendations**](#toc1_4_5_)    
    - [**Summary**](#toc1_4_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [10]:
import sys
import jupyter_black
import warnings
import os

jupyter_black.load()
from IPython.display import display

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import polars as pl
import polars.selectors as cs
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy.stats as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import IsolationForest

from fancyimpute import IterativeImputer
from joblib import Parallel, delayed

from modeling import data_preprocessing, visualisation, feature_engineering

In [11]:
os.chdir("..")

# <a id='toc1_'></a>[The project](#toc0_)

Saha and colleagues collected a large set of breast MRI cases that
were used to detect breast cancer. The dataset also contains detailed
information about the characteristic of the patient and subtype of tumors
that were eventually diagnosed. This information can be used to determine
the best treatment.

Basic image processing was performed to extract features from the MRI
cases which were used to predict the molecular tumor subtypes from MRI.
In this assignment, we will try to predict the estrogen receptor (ER) status
from the MRI image features that were provided.

## <a id='toc1_1_'></a>[1. Descriptive Analysis](#toc0_)

In [12]:
### download Pandas first to check for corruption
df_pandas = pd.read_excel("dataset/intermediate/dataset_MRI_ER_target.xlsx")
### convert pandas dataframe to polars dataframe
data_raw = pl.from_pandas(df_pandas)

### <a id='toc1_1_1_'></a>[1.1 Missing values](#toc0_)

In [13]:
### calculate shares of null values for each column
null_share_dict = data_preprocessing.percentage_of_null_values(data_raw)
null_share_dict

{'Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_1': 4.77,
 'Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_1': 4.77,
 'Grouping_based_mean_of_washin_slope_2D_tumorSlice_Group_1': 4.77,
 'Grouping_based_variance_of_washin_slope_2D_tumorSlice_Group_1': 4.77,
 'Grouping_based_mean_of_washout_slope_2D_tumorSlice_Group_1': 4.77,
 'Grouping_based_variance_of_washout_slope_2D_tumorSlice_Group_1': 4.77,
 'Grouping_based_mean_of_peak_enhancement_slope_3D_tumor_Group_1': 1.52,
 'Grouping_based_variance_of_peak_enhancement_slope_3D_tumor_Group_1': 1.52,
 'Grouping_based_mean_of_washin_slope_3D_tumor_Group_1': 1.52,
 'Grouping_based_variance_of_washin_slope_3D_tumor_Group_1': 1.52,
 'Grouping_based_mean_of_washout_slope_3D_tumor_Group_1': 1.52,
 'Grouping_based_variance_of_washout_slope_3D_tumor_Group_1': 1.52,
 'Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_3': 1.52,
 'Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Gr

**Missing Values Summary**

**Overview**
- **Total number of columns**: 40
- **Highest percentage of null values**: **4.77%**
- **Lowest percentage of null values**: **0.11%**

**Columns with the Highest Null Percentage (4.77%)**
- `Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_1`
- `Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_1`
- `Grouping_based_mean_of_washin_slope_2D_tumorSlice_Group_1`
- `Grouping_based_variance_of_washin_slope_2D_tumorSlice_Group_1`
- `Grouping_based_mean_of_washout_slope_2D_tumorSlice_Group_1`
- `Grouping_based_variance_of_washout_slope_2D_tumorSlice_Group_1`

**Columns with the Lowest Null Percentage (0.11%)**
- `BEVR_Tumor`
- `BEDR1_Tumor`
- `BEDR2_Tumor`
- `MF_Tumor`
- `Volume_cu_mm_Tumor`
- `Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_2`
- `Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_2`
- `Grouping_based_mean_of_washin_slope_2D_tumorSlice_Group_2`
- `Grouping_based_variance_of_washin_slope_2D_tumorSlice_Group_2`
- `Grouping_based_mean_of_washout_slope_2D_tumorSlice_Group_2`
- `Grouping_based_variance_of_washout_slope_2D_tumorSlice_Group_2`

<br>

- Most columns have a **low percentage of missing values**, indicating a relatively complete dataset.
- **Only six columns exceed 4% null values**, which are related to Tumor Enhancement Variation group.
- The majority of columns have a **null value percentage of 1.52% or lower**.


The type of missing data mechanism depends on imaging techniques, tumor biology, data acquisition protocols and other factors. There are three main types of missing data:  
<br> **1. Missing Not at Random (MNAR) – Most Common Cause**:  
Reason: TEV features in breast cancer are highly dependent on tumor vascularity and perfusion. If a tumor has low angiogenesis (poor blood supply), it may not enhance significantly, leading to systematically missing TEV values.  

<br> **2. Missing at Random (MAR) – Imaging Protocol-Dependent**  
Reason: Missing TEV features could be correlated with scanning parameters, contrast administration timing, or segmentation issues rather than tumor characteristics.  

<br> **3. Missing Completely at Random (MCAR) – Less Likely**  
Reason: Some TEV features may be missing due to technical errors, data corruption, or random omissions in the dataset.  

In [14]:
print(
    data_raw.select(
        [col for col, share_null in null_share_dict.items() if share_null > 4]
    ).filter(
        pl.col(
            "Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_1"
        ).is_null()
    )
)

shape: (44, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ Grouping_based ┆ Grouping_based ┆ Grouping_based ┆ Grouping_base ┆ Grouping_base ┆ Grouping_base │
│ _mean_of_peak_ ┆ _variance_of_p ┆ _mean_of_washi ┆ d_variance_of ┆ d_mean_of_was ┆ d_variance_of │
│ enha…          ┆ eak_…          ┆ n_sl…          ┆ _washi…       ┆ hout_s…       ┆ _washo…       │
│ ---            ┆ ---            ┆ ---            ┆ ---           ┆ ---           ┆ ---           │
│ f64            ┆ f64            ┆ f64            ┆ f64           ┆ f64           ┆ f64           │
╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ null           ┆ null           ┆ null           ┆ null          ┆ null          ┆ null          │
│ null           ┆ null           ┆ null           ┆ null          ┆ null          ┆ null          │
│ null           ┆ null           ┆ null           ┆ null          ┆ null   

Since there is a significant difference between the missing and non-missing groups, we assume that the missingness follows the Missing Not at Random (MNAR) or Missing at Random (MAR) pattern.

### <a id='toc1_1_2_'></a>[1.2 Outliers](#toc0_)

In [15]:
df_outliers = data_preprocessing.percentage_of_outliers_iforest(data_raw)
df_outliers

ER,Outlier_Count,Total_Count,Outlier_Percentage
i64,u32,u32,f64
1,7,632,1.11
0,5,229,2.18


In [16]:
### create dataframe to plot bar graph
df_melted = df_outliers.melt(id_vars=["ER"], value_vars=["Outlier_Percentage"]).rename(
    {"variable": "count", "value": "percentage"}
)

### plot bar graph
visualisation.plot_bar_graph(
    df_melted, x="ER", y="percentage", title="Percentage of outliers per ER"
)

We use Isolation Forest algorithm to gidentify outliers. The percentage of outliers varies across ER status, with the ER-negative group having more outliers than the ER-positive group.

### <a id='toc1_1_3_'></a>[1.3 Correlation analysis](#toc0_)

In [17]:
cols_to_drop = data_preprocessing.high_correlation_columns(data_raw)

In [18]:
print(f"The number features with correlation >0.8: {len(cols_to_drop)}")

The number features with correlation >0.8: 341


We can see that 341 out of 529 continuous features exhibit high correlation.

### <a id='toc1_1_4_'></a>[1.4 Target variable](#toc0_)

In [19]:
data_preprocessing.group_by_share_count(data_raw)

ER,Count,Percentage
i64,u32,f64
0,236,25.6
1,686,74.4


In [20]:
### plot bar graph
visualisation.plot_bar_graph(
    data_preprocessing.group_by_share_count(data_raw),
    x="ER",
    y="Percentage",
    title="Percentage of ER status",
)

As we can see target variable is moderately imbalanced.

## <a id='toc1_2_'></a>[2. Data cleaning for train and test sets](#toc0_)

### <a id='toc1_2_1_'></a>[2.1 Split datasets into train and test sets](#toc0_)

Since we have a separate dataset for the test set (test_set.xlsx), we should split the datasets before applying any data processing steps. This helps prevent potential data leakage.

In [21]:
### download test dataset
test_data = pl.read_excel("dataset/raw/test_set.xlsx")

### convert Patient ID into numeric values
test_data = test_data.with_columns(
    test_data["Patient ID"].str.slice(-3, 3).cast(pl.Int32)
)

print(test_data)

shape: (305, 3)
┌────────────┬───────────┬───────────┐
│ Patient ID ┆ y_score_0 ┆ y_score_1 │
│ ---        ┆ ---       ┆ ---       │
│ i32        ┆ f64       ┆ f64       │
╞════════════╪═══════════╪═══════════╡
│ 549        ┆ 0.2646    ┆ 0.7354    │
│ 247        ┆ 0.1412    ┆ 0.8588    │
│ 878        ┆ 0.3014    ┆ 0.6986    │
│ 875        ┆ 0.1114    ┆ 0.8886    │
│ 632        ┆ 0.1778    ┆ 0.8222    │
│ …          ┆ …         ┆ …         │
│ 843        ┆ 0.2052    ┆ 0.7948    │
│ 387        ┆ 0.3176    ┆ 0.6824    │
│ 201        ┆ 0.1976    ┆ 0.8024    │
│ 807        ┆ 0.1322    ┆ 0.8678    │
│ 2          ┆ 0.2038    ┆ 0.7962    │
└────────────┴───────────┴───────────┘


In [ ]:
### split into train and test sets
test_raw_mri_features = data_raw.filter(
    data_raw["Patient ID"].is_in(test_data["Patient ID"])
)
train_raw_mri_features = data_raw.filter(
    ~data_raw["Patient ID"].is_in(test_data["Patient ID"])
)

In [23]:
print(train_raw_mri_features)

shape: (617, 531)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬─────┐
│ Patient ID ┆ F1_DT_POST ┆ F1_DT_POST ┆ F1_DT_POST ┆ … ┆ WashinRate ┆ WashinRat ┆ WashinRat ┆ ER  │
│ ---        ┆ CON (T11=0 ┆ CON (T11=0 ┆ CON (T11=0 ┆   ┆ _map_std_d ┆ e_map_ske ┆ e_map_kur ┆ --- │
│ i64        ┆ .05,T12=0. ┆ .05,T12=0. ┆ .02,T12=0. ┆   ┆ ev_tissue_ ┆ wness_tis ┆ tosis_tis ┆ i64 │
│            ┆ 5)         ┆ 1)         ┆ 5)         ┆   ┆ Po…        ┆ sue_P…    ┆ sue_P…    ┆     │
│            ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆     │
│            ┆ f64        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ f64       ┆     │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═════╡
│ 1          ┆ 1.0        ┆ 0.120721   ┆ 0.530395   ┆ … ┆ 20.347506  ┆ 1.62587   ┆ 11.406955 ┆ 0   │
│ 3          ┆ 0.174775   ┆ 0.062051   ┆ 0.06991    ┆ … ┆ 129.252343 ┆ 1.

In [24]:
print(test_raw_mri_features)

shape: (305, 531)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬─────┐
│ Patient ID ┆ F1_DT_POST ┆ F1_DT_POST ┆ F1_DT_POST ┆ … ┆ WashinRate ┆ WashinRat ┆ WashinRat ┆ ER  │
│ ---        ┆ CON (T11=0 ┆ CON (T11=0 ┆ CON (T11=0 ┆   ┆ _map_std_d ┆ e_map_ske ┆ e_map_kur ┆ --- │
│ i64        ┆ .05,T12=0. ┆ .05,T12=0. ┆ .02,T12=0. ┆   ┆ ev_tissue_ ┆ wness_tis ┆ tosis_tis ┆ i64 │
│            ┆ 5)         ┆ 1)         ┆ 5)         ┆   ┆ Po…        ┆ sue_P…    ┆ sue_P…    ┆     │
│            ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆     │
│            ┆ f64        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ f64       ┆     │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═════╡
│ 2          ┆ 1.0        ┆ 0.129546   ┆ 0.485217   ┆ … ┆ 83.909561  ┆ 0.251498  ┆ 5.659428  ┆ 0   │
│ 4          ┆ 0.086546   ┆ 0.045111   ┆ 0.034619   ┆ … ┆ 69.164227  ┆ 1.

In [25]:
## save train and test datasets
train_raw_mri_features.write_excel("dataset/intermediate/train.xlsx")
test_raw_mri_features.write_excel("dataset/intermediate/test.xlsx")

### <a id='toc1_2_2_'></a>[2.2 Handling missing values](#toc0_)

#### <a id='toc1_2_2_1_'></a>[2.2.1 Imputation for train set](#toc0_)

In [26]:
train_raw = pl.read_excel("dataset/intermediate/train.xlsx")

In [27]:
### check the percentage of missing values
data_preprocessing.percentage_of_null_values(train_raw)

{'Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_1': 4.21,
 'Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_1': 4.21,
 'Grouping_based_mean_of_washin_slope_2D_tumorSlice_Group_1': 4.21,
 'Grouping_based_variance_of_washin_slope_2D_tumorSlice_Group_1': 4.21,
 'Grouping_based_mean_of_washout_slope_2D_tumorSlice_Group_1': 4.21,
 'Grouping_based_variance_of_washout_slope_2D_tumorSlice_Group_1': 4.21,
 'Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_3': 1.62,
 'Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_3': 1.62,
 'Grouping_based_mean_of_washin_slope_2D_tumorSlice_Group_3': 1.62,
 'Grouping_based_variance_of_washin_slope_2D_tumorSlice_Group_3': 1.62,
 'ASD_Tumor': 1.3,
 'Grouping_based_mean_of_peak_enhancement_slope_3D_tumor_Group_3': 1.3,
 'Grouping_based_variance_of_peak_enhancement_slope_3D_tumor_Group_3': 1.3,
 'Grouping_based_mean_of_washin_slope_3D_tumor_Group_3': 1.3,
 'Grouping_based_variance_of_

Since the proportion of missing values is not large and we have many features, we can impute the missing values using the MICE algorithm.

In [28]:
### Convert to NumPy
data_np = train_raw.to_numpy()

### Find optimal max_iter
optimal_iter, imputed_data = data_preprocessing.find_optimal_max_iter_fancyimpute(
    data_np, max_iters=20
)
print(f"Optimal max_iter: {optimal_iter}")

Checking Convergence:   0%|          | 0/19 [00:00<?, ?it/s]

Iteration 2: Mean absolute change = 0.000000
Converged at max_iter = 2
Optimal max_iter: 2


In [29]:
### convert ndarray to polars dataframe
train_processed_full = pl.from_numpy(imputed_data)

### check the percentage of missing values
data_preprocessing.percentage_of_null_values(train_processed_full)

There are no missing values in the dataset


In [30]:
### rename columns to have original column names
train_processed_full = train_processed_full.rename(
    dict(zip(train_processed_full.columns, train_raw.columns))
)

In [32]:
### save the train dataset
train_processed_full.write_excel("dataset/processed/train_processed_full.xlsx")

#### <a id='toc1_2_2_2_'></a>[2.2.2 Imputation for test set](#toc0_)

In [33]:
test_raw = pl.read_excel("dataset/intermediate/test.xlsx")

In [34]:
### check the percentage of missing values
data_preprocessing.percentage_of_null_values(test_raw)

{'Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_1': 5.9,
 'Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_1': 5.9,
 'Grouping_based_mean_of_washin_slope_2D_tumorSlice_Group_1': 5.9,
 'Grouping_based_variance_of_washin_slope_2D_tumorSlice_Group_1': 5.9,
 'Grouping_based_mean_of_washout_slope_2D_tumorSlice_Group_1': 5.9,
 'Grouping_based_variance_of_washout_slope_2D_tumorSlice_Group_1': 5.9,
 'Grouping_based_mean_of_peak_enhancement_slope_3D_tumor_Group_1': 2.3,
 'Grouping_based_variance_of_peak_enhancement_slope_3D_tumor_Group_1': 2.3,
 'Grouping_based_mean_of_washin_slope_3D_tumor_Group_1': 2.3,
 'Grouping_based_variance_of_washin_slope_3D_tumor_Group_1': 2.3,
 'Grouping_based_mean_of_washout_slope_3D_tumor_Group_1': 2.3,
 'Grouping_based_variance_of_washout_slope_3D_tumor_Group_1': 2.3,
 'Grouping_based_mean_of_peak_enhancement_slope_2D_tumorSlice_Group_3': 1.31,
 'Grouping_based_variance_of_peak_enhancement_slope_2D_tumorSlice_Group_3': 1.31

Since the proportion of missing values is not large and we have many features, we can impute the missing values using the MICE algorithm.

In [35]:
### Convert to NumPy
data_np = test_raw.to_numpy()

### Find optimal max_iter
optimal_iter, imputed_data = data_preprocessing.find_optimal_max_iter_fancyimpute(
    data_np, max_iters=20
)
print(f"Optimal max_iter: {optimal_iter}")

Checking Convergence:   0%|          | 0/19 [00:00<?, ?it/s]

Iteration 2: Mean absolute change = 0.000000
Converged at max_iter = 2
Optimal max_iter: 2


In [36]:
### convert ndarray to polars dataframe
test_processed_full = pl.from_numpy(imputed_data)

### check the percentage of missing values
data_preprocessing.percentage_of_null_values(test_processed_full)

There are no missing values in the dataset


In [37]:
### rename columns to have original column names
test_processed_full = test_processed_full.rename(
    dict(zip(test_processed_full.columns, test_raw.columns))
)

In [38]:
### save the test dataset
test_processed_full.write_excel("dataset/processed/test_processed_full.xlsx")

## <a id='toc1_3_'></a>[3. Data processing approaches](#toc0_)

In my modeling I used 4 approaches:  
1. Approach 1 (Baseline):
To establish a straightforward reference model and assess the impact of subsequent preprocessing or feature engineering steps.  

2. Approach 2 (Correlation-Based Feature Selection & Outlier Removal):
To reduce redundancy in highly correlated MRI features (minimizing multicollinearity and overfitting) and remove anomalous data points that could skew model performance.  

3. Approach 3 (Statistical Distribution–Based Selection via KS Test):
To identify features with significantly different distributions across ER statuses, particularly suitable for non-normal radiomic data and robust to unequal group sizes.  

4. Approach 4 (Non-Linear Dimensionality Reduction + Robust Scaling):
To handle skewed, non-Gaussian features via outlier-resistant scaling and to capture complex, non-linear relationships using UMAP, which balances local and global data structure.

### <a id='toc1_3_1_'></a>[3.1 Approach 1: Baseline Model (Full Data)](#toc0_)

In the initial approach, no data processing or feature engineering is performed on the datasets. This dataset serves as a benchmark for comparison.

In [39]:
train_processed_full = pl.read_excel("dataset/processed/train_processed_full.xlsx")
test_processed_full = pl.read_excel("dataset/processed/test_processed_full.xlsx")

### <a id='toc1_3_2_'></a>[3.2 Approach 2: Correlation-Based Feature Selection & Outlier Removal](#toc0_)

**Steps used in this approach:**

- Feature selection: Remove features with high correlation.
- Feature engineering: Remove outliers.
- Saling: No scaling is performed.


#### <a id='toc1_3_2_1_'></a>[3.2.1 Correlation-Based Feature Selection for train and test sets separately](#toc0_)

In [40]:
### 1. drop highly correlated columns for train dataset
cols_to_drop = data_preprocessing.high_correlation_columns(
    train_processed_full, col_to_drop=["Patient ID", "ER"], threshold=0.8
)
print(
    f"The number features with correlation >0.8 for train dataset: {len(cols_to_drop)}"
)
train_processed_approach_2 = train_processed_full.drop(cols_to_drop)

The number features with correlation >0.8 for train dataset: 359


In [41]:
### 2. drop highly correlated columns for test dataset
print(
    f"The number features with correlation >0.8 for test dataset: {len(cols_to_drop)}"
)
test_processed_approach_2 = test_processed_full.drop(cols_to_drop)

The number features with correlation >0.8 for test dataset: 359


#### <a id='toc1_3_2_2_'></a>[3.2.2 Outlier Removal for train set](#toc0_)

We use Isolation Forest (IF) algorithm to detect outliers for several reasons: 
- Isolation Forest doesn't rely on distance metrics but instead recursively partitions data using random splits. This makes it more scalable and efficient for high-dimensional datasets like radiogenomic features.
- MRI-derived radiomic features (like texture, intensity, shape) can have significant correlations and noise. Isolation Forest (IF) algorithm is robust to noise and feature redundancy.
- Isolation Forest does not assume any specific data distribution, making it well-suited for handling heterogeneous data.

In [42]:
### get percentage of outliers across ER status for train dataset
print(f"Outlier percentage for train dataset")
data_preprocessing.percentage_of_outliers_iforest(train_processed_approach_2)

Outlier percentage for train dataset


ER,Outlier_Count,Total_Count,Outlier_Percentage
i64,u32,u32,f64
0,4,158,2.53
1,1,459,0.22


In [43]:
### 1. Remove outliers for train datasets
train_processed_approach_2, df_outliers = (
    data_preprocessing.remove_outliers_isolation_forest(train_processed_approach_2)
)

In [44]:
train_processed_approach_2.write_excel(
    "dataset/processed/train_processed_approach_2.xlsx"
)
test_processed_approach_2.write_excel(
    "dataset/processed/test_processed_approach_2.xlsx"
)

#### <a id='toc1_3_2_3_'></a>[3.2.3 Statistical analysis](#toc0_)

I will join train_processed_approach_2 and test_processed_approach_2 datasets to perform statistical analysis.

In [45]:
data_processed_approach_2 = pl.concat(
    [train_processed_approach_2, test_processed_approach_2]
)

##### <a id='toc1_3_2_3_1_'></a>[1) The proportion of data points per ERs](#toc0_)

In [46]:
data_preprocessing.group_by_count_join_train_test(
    train_processed_approach_2, test_processed_approach_2
)

ER,Train_Count,Train_Percentage,Test_Count,Test_Percentage
i64,u32,f64,u32,f64
0,154,25.16,78,25.57
1,458,74.84,227,74.43


In [47]:
df_melted = (
    data_preprocessing.group_by_count_join_train_test(
        train_processed_approach_2, test_processed_approach_2
    )
    .melt(id_vars=["ER"], value_vars=["Train_Percentage", "Test_Percentage"])
    .rename({"variable": "category", "value": "percentage"})
)
visualisation.plot_bar_graph(
    df_melted,
    x="ER",
    y="percentage",
    title="The proportion of data points per ERs",
    color="category",
)

The proportions of data points for each ER status are nearly identical, indicating that our dataset is properly split into training and test sets.

##### <a id='toc1_3_2_3_2_'></a>[2) TOP-10 significant features](#toc0_)

In order to identify TOP-10 significant features I use Point-Biserial Correlation Coefficient.

Point-Biserial Correlation Coefficient (r_pb)
The Point-Biserial Correlation Coefficient (r_pb) measures the correlation between a continuous variable and a binary variable. It is computed using the formula:

$$
r_{pb} = \frac{\bar{X_1} - \bar{X_0}}{s} \times \sqrt{\frac{n_1 n_0}{n(n-1)}}
$$

Where:
- $ \bar{X_1} $ = Mean of the continuous feature for the group where the binary target is 1.  
- $ \bar{X_0} $ = Mean of the continuous feature for the group where the binary target is 0.  
- $ s $ = Standard deviation of the continuous feature across all samples.  
- $ n_1 $ = Number of samples where the binary target is 1.  
- $ n_0 $ = Number of samples where the binary target is 0.  
- $ n = n_1 + n_0 $ = Total number of samples.


In [48]:
top_10_point_biserial_features, corr_data = (
    data_preprocessing.point_biserial_correlation(data_processed_approach_2)
)
print(top_10_point_biserial_features)

shape: (10, 2)
┌───────────────────────────────────┬─────────────┐
│ feature                           ┆ correlation │
│ ---                               ┆ ---         │
│ str                               ┆ f64         │
╞═══════════════════════════════════╪═════════════╡
│ WashinRate_map_Cluster_Prominenc… ┆ 0.190526    │
│ Volume_cu_mm_Tumor                ┆ 0.186833    │
│ Grouping_based_proportion_of_tum… ┆ 0.174255    │
│ WashinRate_map_information_measu… ┆ 0.16424     │
│ WashinRate_map_information_measu… ┆ 0.157035    │
│ TumorMajorAxisLength_mm           ┆ 0.150338    │
│ WashinRate_map_skewness_tumor     ┆ 0.147612    │
│ WashinRate_map_Autocorrelation_t… ┆ 0.142321    │
│ WashinRate_map_Contrast_tumor     ┆ 0.134552    │
│ Inf_mea_of_corr1_Tumor            ┆ 0.125213    │
└───────────────────────────────────┴─────────────┘


**Summary**  
The features mostly correlated with **ER status** can be categorized into 4 main groups:

**Tumor Enhancement Texture:**
- WashinRate_map_Cluster_Prominence_tumor
- WashinRate_map_information_measure_correlation2_tumor
- WashinRate_map_information_measure_correlation1_tumor
- WashinRate_map_Autocorrelation_tumor

**Tumor Size and Morphology:**
- Volume_cu_mm_Tumor
- TumorMajorAxisLength
- WashinRate_map_Contrast_tumor
- Inf_mea_of_corr1_Tumor

**Tumor Enhancement:**
- Grouping_based_proportion_of_tumor_voxels_3D_tumor_Group_1

**Tumor Enhancement Variation:**
- WashinRate_map_skewness_tumor

In [49]:
### 1. create dataset to plot box plots
df_top10_features = data_processed_approach_2.select(
    top_10_point_biserial_features.select("feature").to_series().to_list() + ["ER"]
)

### 2. rename columns
df_top10_features = df_top10_features.rename(
    {col: f"col_{i+1}" for i, col in enumerate(df_top10_features.columns[:-1])}
)

### 3. plot box plots
visualisation.plot_10_box_plot(df_top10_features)

Based on the boxplots, the binary output (ER status: 0 or 1) **is not perfectly separable**, but some features provide a degree of separation between the two classes. Here's why:

**1. Moderately Distinguishable Features:**
Some features show a noticeable difference in medians and interquartile ranges (IQRs), indicating a level of separability: WashinRate_map_Cluster_Prominence_tumor, Grouping_based_proportion_of_tumor_voxels_3D_tumor_Group_1,WashinRate_map_information_measure_correlation2_tumor,WashinRate_map_information_measure_correlation1_tumor,
WashinRate_map_Autocorrelation_tumor.

**2. Poorly Separating Features:**
Volume_cu_mm_Tumor and TumorMajorAxisLength_mm: These have high numbers of outliers in both categories, making separation difficult.
WashinRate_map_Cluster_Shade_tumor: The spread is very large, with overlapping distributions between ER=0 and ER=1.

**processed Conclusion:**
The dataset is not linearly separable, as many features have overlapping distributions between the two ER groups.

In [50]:
### 3. plot kde plots
visualisation.plot_10_kde_plots(df_top10_features)

**Summary**  

Based on the KDE plots, the binary output (ER status: 0 or 1) **is not perfectly separable**, but some features provide moderate separation.

**processed Conclusion:**
- The binary classes are not fully separable.
- Some features (WashinRate_map_Cluster_Prominence_tumor, WashinRate_map_information_measure_correlation2_tumor, WashinRate_map_Cluster_Shade_tumor) provide good separation, but most features seem to have very moderate discriminatory power. 

In [51]:
### 1. Plot umap components for train dataset
df_umap_approach_2 = feature_engineering.run_umap(
    data_processed_approach_2, n_components=2, n_trials=20, save=False
)
df_plot = df_umap_approach_2.drop("Patient ID").rename(
    {"UMAP_0": "UMAP Component 1", "UMAP_1": "UMAP Component 2"}
)
visualisation.plot_scatter_graph(
    df_plot,
    x="UMAP Component 1",
    y="UMAP Component 2",
    color="ER",
    title="UMAP Projection of train data for approach 2",
)

Best UMAP Parameters Saved: {'n_neighbors': 5, 'min_dist': 0.043350107354230075, 'spread': 1.033211283066237, 'metric': 'euclidean', 'negative_sample_rate': 1, 'learning_rate': 0.41999907442072587}


**Summary:**

The binary output (ER status) **is not separable** based on the UMAP projection. This suggests that the selected features and the UMAP transformation may not be sufficient for effective classification in this lower-dimensional space. 

In [52]:
### 1. Plot tsne components for train dataset
df_tsne_approach_2 = feature_engineering.run_tsne(
    data_processed_approach_2,
    id_column="Patient ID",
    target_column="ER",
    n_components=2,
)
df_plot = df_tsne_approach_2.drop("Patient ID")
visualisation.plot_scatter_graph(
    df_plot,
    x="tSNE_0",
    y="tSNE_1",
    color="ER",
    title="tSNE Projection of train data for approach 2",
)

**Summary:**

The binary target **shows some degree of separation** in t-SNE space, but it is not fully distinct.

**1. t-SNE:**
- Shows more structured clustering, where ER = 1 (yellow) and ER = 0 (dark blue) are somewhat grouped, especially in certain regions. While there is still overlap, we see more localized clusters of the same class.

**2. UMAP:**
- The two classes appear more evenly mixed throughout the space, with no clear separation or clustering. 

**t-SNE shows better separation because there are more identifiable clusters where one class dominates.**

### <a id='toc1_3_3_'></a>[3.3 Approach 3: Statistical Distribution-Based Feature Selection ](#toc0_)

**Steps used in this approach:**

- Feature selection: Select features based on distribution differences between the binary labels.
- Feature engineering: No outlier removal.
- Sacaling: No scaling.

#### <a id='toc1_3_3_1_'></a>[3.3.1 Statistical Distribution-Based Feature Selection ](#toc0_)

We apply the two-sample KS test to identify distribution differences between the binary labels and select features with significant test results. We use the two-sample KS test based on the following assumptions:  
1. Independent and Identically Distributed (i.i.d.) data:
The observations in the samples are independent and drawn from the same distribution.
2. Continuous Data:
The observations have continuous distributions.
3. Non-Parametric Nature:
The KS test does not assume any specific form for the distribution.

The reasons to use the KS test:
1. MRI features are mostly non-normal:
Since the KS test is non-parametric, it works well without assuming normality.
2. Identifies Significant Distributional Differences:
If MRI-derived features have distinct distributions for patients across ER status, the KS test can confirm that these differences are statistically meaningful.
3. Handles Unequal Sample Sizes:
Unlike parametric tests like t-tests, KS works well even when group sizes are different.
4. Applicable for High-Dimensional Data.
5. Sensitivity to Differences:
The test is sensitive to differences in both the shape and location of the distributions (e.g., differences in means, variances, or skewness).

In [53]:
### 1. drop features with non-significant distribution difference for train dataset
selected_features_train, cols_to_drop = data_preprocessing.ks_test_feature_selection(
    train_processed_full
)
print(
    f"The number of features with non significant test results for train dataset: {len(cols_to_drop)}"
)
train_processed_approach_3 = train_processed_full.drop(cols_to_drop)

The number of features with non significant test results for train dataset: 380


In [54]:
### 1. drop features with non-significant distribution difference for test dataset
print(
    f"The number of features with non significant test results for test dataset: {len(cols_to_drop)}"
)
test_processed_approach_3 = test_processed_full.drop(cols_to_drop)

The number of features with non significant test results for test dataset: 380


In [55]:
train_processed_approach_3.write_excel(
    "dataset/processed/train_processed_approach_3.xlsx"
)
test_processed_approach_3.write_excel(
    "dataset/processed/test_processed_approach_3.xlsx"
)

#### <a id='toc1_3_3_2_'></a>[3.3.2 Statistical analysis](#toc0_)

I will join train_processed_approach_3 and test_processed_approach_3 datasets to perform statistical analysis.

In [56]:
data_processed_approach_3 = pl.concat(
    [train_processed_approach_3, test_processed_approach_3]
)

##### <a id='toc1_3_3_2_1_'></a>[1) The proportion of data points per ERs](#toc0_)

In [57]:
data_preprocessing.group_by_count_join_train_test(
    train_processed_approach_3, test_processed_approach_3
)

ER,Train_Count,Train_Percentage,Test_Count,Test_Percentage
i64,u32,f64,u32,f64
0,158,25.61,78,25.57
1,459,74.39,227,74.43


In [58]:
df_melted = (
    data_preprocessing.group_by_count_join_train_test(
        train_processed_approach_3, test_processed_approach_3
    )
    .melt(id_vars=["ER"], value_vars=["Train_Percentage", "Test_Percentage"])
    .rename({"variable": "category", "value": "percentage"})
)
visualisation.plot_bar_graph(
    df_melted,
    x="ER",
    y="percentage",
    title="The proportion of data points per ERs",
    color="category",
)

The proportions of data points for each ER status are nearly identical, indicating that our dataset is properly split into training and test sets.

##### <a id='toc1_3_3_2_2_'></a>[2) TOP-10 statistically significant features](#toc0_)

Selection of top-10 features using KS test:
1. Sort all features by ascending p-value (the smallest p-value means the most significant difference).
2. Features with the smallest p-values are the most discriminative between the two classes.
3. Pick the top 10 features with the lowest p-values.

In [59]:
selected_features, cols_to_drop = data_preprocessing.ks_test_feature_selection(
    data_processed_approach_3
)

In [60]:
top_10_ks_test_features = selected_features.limit(10)
print(top_10_ks_test_features)

shape: (10, 2)
┌───────────────────────────────────┬────────────┐
│ feature                           ┆ p_value    │
│ ---                               ┆ ---        │
│ str                               ┆ f64        │
╞═══════════════════════════════════╪════════════╡
│ SER_Washout_tumor_vol_cu_mm       ┆ 6.8507e-12 │
│ SER_Partial_tumor_vol_cu_mm       ┆ 3.1453e-10 │
│ Grouping_based_proportion_of_tum… ┆ 4.6965e-9  │
│ WashinRate_map_Cluster_Prominenc… ┆ 1.0305e-8  │
│ SER_Total_tumor_vol_cu_mm         ┆ 4.0329e-8  │
│ Volume_cu_mm_Tumor                ┆ 3.2092e-7  │
│ SER_map_mean_tumor                ┆ 4.2390e-7  │
│ Grouping_based_proportion_of_tum… ┆ 0.000001   │
│ WashinRate_map_inverse_differenc… ┆ 0.000002   │
│ TumorMajorAxisLength_mm           ┆ 0.000002   │
└───────────────────────────────────┴────────────┘


**Summary**  
All 10 features have extremely low p-values (< 0.00001), indicating highly significant distributional differences.

**Tumor Enhancement:**
- SER_Washout_tumor_vol_cu_mm
- SER_Partial_tumor_vol_cu_mm
- Grouping_based_proportion_of_tumor_voxels_3D_tumor_Group_1
- SER_Total_tumor_vol_cu_mm
- SER_map_mean_tumor
- Grouping_based_proportion_of_tumor_voxels_2D_tumorSlice_Group_1

**Tumor Enhancement Texture:**
- WashinRate_map_Cluster_Prominence_tumor
- WashinRate_map_inverse_difference_moment_normalized_tumor

**Tumor Size and Morphology:**
- Volume_cu_mm_Tumor
- TumorMajorAxisLength

In [61]:
### 1. create dataset to plot box plots
df_top10_features = data_processed_approach_3.select(
    top_10_ks_test_features.select("feature").to_series().to_list() + ["ER"]
)

### 2. rename columns
df_top10_features = df_top10_features.rename(
    {col: f"col_{i+1}" for i, col in enumerate(df_top10_features.columns[:-1])}
)

### 3. plot box plots
visualisation.plot_10_box_plot(df_top10_features)

**Summary:**

Most features **show moderate separation** between ER = 0 and ER = 1, which may indicate their importance in distinguishing between the groups.

In [62]:
### 3. plot box plots
visualisation.plot_10_kde_plots(df_top10_features)

**Summary**  

Based on the KDE plots, the binary output (ER status: 0 or 1) **is not perfectly separable**, but some features provide moderate separation.

In [63]:
### 1. Plot umap components for train dataset
df_umap_approach_3 = feature_engineering.run_umap(
    data_processed_approach_3, n_components=2, n_trials=20, save=False
)
df_plot = df_umap_approach_3.drop("Patient ID").rename(
    {"UMAP_0": "UMAP Component 1", "UMAP_1": "UMAP Component 2"}
)
visualisation.plot_scatter_graph(
    df_plot,
    x="UMAP Component 1",
    y="UMAP Component 2",
    color="ER",
    title="UMAP Projection of train data for approach 3",
)

Best UMAP Parameters Saved: {'n_neighbors': 6, 'min_dist': 0.24315748927031564, 'spread': 0.6100467440027604, 'metric': 'euclidean', 'negative_sample_rate': 4, 'learning_rate': 0.10358949274338423}


**Summary:**

The binary target (ER status) **is not clearly separable** in this UMAP representation. This suggests that even with the top 10 features selected by the Kolmogorov-Smirnov test, the two classes still have considerable overlap.

In [64]:
### 1. Plot tsne components for train dataset
df_tsne_approach_3 = feature_engineering.run_tsne(
    data_processed_approach_3,
    id_column="Patient ID",
    target_column="ER",
    n_components=2,
)
df_plot = df_tsne_approach_3.drop("Patient ID")
visualisation.plot_scatter_graph(
    df_plot,
    x="tSNE_0",
    y="tSNE_1",
    color="ER",
    title="tSNE Projection of train data for approach 3",
)

**Summary:**

The t-SNE visualization confirms the UMAP findings—the binary target (ER status) **is not easily separable** based on the top 10 features selected. This indicates that the chosen features may not be sufficient for classification

### <a id='toc1_3_4_'></a>[3.4 Approach 4: Non-Linear Dimensionality Reduction & Scaling](#toc0_)

**Steps used in this approach:**

- Scaling: Yes (Robust Scaler)
- Feature selection: Non-linear dimensionality reduction (UMAP)
- Feature engineering: No outlier removal

#### <a id='toc1_3_4_1_'></a>[3.4.1 Scaling: Robust Scaler for train and test sets separately](#toc0_)

**Reasons for Using Robust Scaler in Our Analysis:**

**Handling Outliers:**  
RobustScaler scales features using the median and interquartile range (IQR), making it more resistant to outliers compared to Standardization and MinMaxScaling. Since our dataset contains outliers, this approach is more suitable.

**Non-Normal Distribution:**  
Most features in our dataset do not follow a normal distribution. Unlike standardization methods that assume a Gaussian distribution, RobustScaler performs well on non-normally distributed data.

**Improving UMAP Segmentation:**  
RobustScaler ensures that all features are on the same scale, which benefits models like UMAP that rely on distance measurements for segmentation.

**Formula:**

$X_{\text{scaled}} = \frac{X - \text{median}(X)}{\text{IQR}(X)}$

Where:

- $\text{median}(X)$ is the median of the feature.  
- $\text{IQR}(X) = Q_3 - Q_1$ is the interquartile range, where:  
  - $Q_3$ is the 75th percentile.  
  - $Q_1$ is the 25th percentile.  

This ensures that the transformation is centered around zero and is less sensitive to extreme values.  Robust Scaler ensures most values are within [-1,1], while the outlier is controlled but not removed.


In [65]:
### 1. Robust Scaling for train dataset
train_processed_approach_4 = feature_engineering.robust_scaler(
    train_processed_full, id_column="Patient ID", target_column="ER"
)

In [66]:
### 1. Robust Scaling for test dataset
test_processed_approach_4 = feature_engineering.robust_scaler(
    test_processed_full, id_column="Patient ID", target_column="ER"
)

#### <a id='toc1_3_4_2_'></a>[3.4.2 Dimensionality reduction: UMAP for train and test sets separately](#toc0_)

**UMAP (Uniform Manifold Approximation and Projection)** is a non-linear dimensionality reduction technique. It is particularly effective at preserving both global and local structures in datasets.

**Reasons for Using UMAP in Our Analysis:**

1. Preservation of Local and Global Structure:
Unlike t-SNE, which primarily captures local neighborhoods, UMAP maintains a balance between local and global relationships, providing a more comprehensive representation of the data.

2. Scalability:
UMAP is computationally more efficient than t-SNE, making it well-suited for handling our dataset.

3. Non-Linear Dimensionality Reduction:
Given that our dataset contains numerous features with potential non-linear relationships to ER status, UMAP is an appropriate technique for capturing these complex patterns.

In [69]:
### 1.UMAP for train set
train_processed_approach_4 = feature_engineering.run_umap(
    train_processed_approach_4,
    id_column="Patient ID",
    target_column="ER",
    n_components=100,
    n_trials=50,
)

Best UMAP Parameters Saved: {'n_neighbors': 5, 'min_dist': 0.006845194169693487, 'spread': 0.6788613611305662, 'metric': 'euclidean', 'negative_sample_rate': 14, 'learning_rate': 0.15699563401818603}


In [70]:
### save train_processed_approach_4 dataset
train_processed_approach_4.write_excel(
    "dataset/processed/approach_4/train_processed_approach_4.xlsx"
)

In [71]:
### 1.UMAP for test set using best parameters obtained from fitting the train set
test_processed_approach_4 = feature_engineering.run_umap_test_set(
    test_processed_approach_4,
    id_column="Patient ID",
    target_column="ER",
    n_components=100,
)

Test Set Trustworthiness Score: 0.8612949163768836


In [72]:
### save test_processed_approach_4 dataset
test_processed_approach_4.write_excel(
    "dataset/processed/approach_4/test_processed_approach_4.xlsx"
)

**Summary**:

1. We obtained a Trustworthiness Score for test set of 0.865 means that 86.5% of the local neighborhood relationships are retained after UMAP transformation.
2. This is an acceptable score, suggesting that the dimensionality reduction process minimally distorts the data structure.
3. UMAP is effectively preserving important local relationships, making the low-dimensional representation reliable.

#### <a id='toc1_3_4_3_'></a>[3.3.2 Statistical analysis](#toc0_)

I will join train_processed_approach_4 and test_processed_approach_4 datasets to perform statistical analysis.

In [73]:
data_processed_approach_4 = pl.concat(
    [train_processed_approach_4, test_processed_approach_4]
)

##### <a id='toc1_3_4_3_1_'></a>[1) TOP-10 significant components](#toc0_)

In order to identify TOP-10 significant features I use Point-Biserial Correlation Coefficient.

Point-Biserial Correlation Coefficient (r_pb)
The Point-Biserial Correlation Coefficient (r_pb) measures the correlation between a continuous variable and a binary variable. It is computed using the formula:

$$
r_{pb} = \frac{\bar{X_1} - \bar{X_0}}{s} \times \sqrt{\frac{n_1 n_0}{n(n-1)}}
$$

Where:
- $ \bar{X_1} $ = Mean of the continuous feature for the group where the binary target is 1.  
- $ \bar{X_0} $ = Mean of the continuous feature for the group where the binary target is 0.  
- $ s $ = Standard deviation of the continuous feature across all samples.  
- $ n_1 $ = Number of samples where the binary target is 1.  
- $ n_0 $ = Number of samples where the binary target is 0.  
- $ n = n_1 + n_0 $ = Total number of samples.


In [74]:
top_10_point_biserial_features, corr_data = (
    data_preprocessing.point_biserial_correlation(data_processed_approach_4)
)
print(top_10_point_biserial_features)

shape: (10, 2)
┌─────────┬─────────────┐
│ feature ┆ correlation │
│ ---     ┆ ---         │
│ str     ┆ f64         │
╞═════════╪═════════════╡
│ UMAP_30 ┆ 0.094754    │
│ UMAP_99 ┆ 0.080645    │
│ UMAP_4  ┆ 0.080183    │
│ UMAP_31 ┆ 0.068831    │
│ UMAP_71 ┆ 0.067574    │
│ UMAP_57 ┆ 0.058774    │
│ UMAP_56 ┆ 0.056196    │
│ UMAP_11 ┆ 0.054125    │
│ UMAP_35 ┆ 0.049029    │
│ UMAP_23 ┆ 0.04878     │
└─────────┴─────────────┘


**Summary:**

The summary of the data shows the top 10 features (UMAP components) with the highest correlation to the target variable. The correlation values range from 0.0730 (UMAP_86) to 0.0441 (UMAP_59). **These correlations are relatively weak**, suggesting that none of the UMAP features have a strong linear relationship with the target. 

In [76]:
### 1. create dataset to plot box plots
df_top10_features = data_processed_approach_4.select(
    top_10_point_biserial_features.select("feature").to_series().to_list() + ["ER"]
)

### 3. plot box plots
visualisation.plot_10_box_plot(df_top10_features)

**Summary:**

Based on the boxplots, the binary output (ER status: 0 or 1) **is not separable**.

## <a id='toc1_4_'></a>[4. Conclusions](#toc0_)

**Overall Conclusions of Used Approaches**

The exploratory data analysis (EDA) and feature selection approaches
applied to the MRI-derived radiomic features and ER status binary target
have provided valuable insights into the dataset's structure, feature
importance, and potential challenges for modeling. Below is a detailed
summary of the conclusions drawn from each approach, along with general
observations.

### <a id='toc1_4_1_'></a>[**1. Baseline (Approach 1)**](#toc0_)

-   **Purpose:**

    -   This approach serves as an unprocessed benchmark, providing a
        baseline for comparison with more advanced methods.

    -   No data cleaning, feature selection, or outlier removal was
        performed.

    -   The baseline model highlights the raw performance of the dataset
        without any preprocessing, which is useful for evaluating the
        impact of subsequent feature engineering and selection steps.

### <a id='toc1_4_2_'></a>[**2. Correlation-Based Selection & Outlier Removal (Approach 2)**](#toc0_)

-   **Steps:**

    -   Removed 359 highly correlated features (r > 0.8) to address
        multicollinearity and overfitting.

    -   Identified and removed 5 outliers using the Isolation Forest
        algorithm.

-   **Key Insights:**

    -   **Feature Importance:** The top 10 features identified are
        primarily related to:

        1.  **Tumor Enhancement Texture:** Features like
            *WashinRate_map_Cluster_Prominence_tumor* and
            *WashinRate_map_information_measure_correlation2_tumor*
            highlight the importance of texture-based metrics.

        2.  **Tumor Size and Morphology:** Features such as
            *Volume_cu_mm_Tumor* and *TumorMajorAxisLength* emphasize
            the role of tumor size and shape in distinguishing ER
            status.

        3.  **Tumor Enhancement Variation:** Metrics like
            *WashinRate_map_skewness_tumor* suggest that variations in
            tumor enhancement patterns are also significant.

        4.  **Tumor Enhancement:**
            *Grouping_based_proportion_of_tumor_voxels_3D_tumor_Group_1*.

    <!-- -->

    -   **Visualizations:**

        -   **t-SNE:** Shows slightly better clustering compared to
            UMAP, with localized regions where one class dominates.
            However, significant overlap remains, indicating that ER
            status is not easily separable.

        -   **UMAP:** The classes are more evenly mixed, with no clear
            separation.

<!-- -->

-   **Conclusion:** While this approach reduces redundancy and
    identifies key features, the moderate overlap in visualizations
    suggests that additional feature engineering or non-linear modeling
    techniques may be necessary.

### <a id='toc1_4_3_'></a>[**3. Statistical Distribution-Based Selection (Approach 3)**](#toc0_)

-   **Steps:**

    -   Applied the two-sample Kolmogorov-Smirnov (KS) test to identify
        features with significant distributional differences between ER
        status groups.

    -   Selected the top 10 features based on the lowest p-values (<
        0.00001).

-   **Key Insights:**

    -   **Feature Importance:** The top 10 features are consistent with
        Approach 2 in highlighting:

        1.  **Tumor Enhancement:** Features like
            *SER_Washout_tumor_vol_cu_mm* and *SER_map_mean_tumor*
            emphasize the importance of SER-derived volumetric and
            intensity metrics.

        2.  **Tumor Size and Morphology:** Metrics such as
            *Volume_cu_mm_Tumor* and *TumorMajorAxisLength* again appear
            as significant.

        3.  **Tumor Enhancement Texture:** Features like
            *WashinRate_map_Cluster_Prominence_tumor* and
            *WashinRate_map_inverse_difference_moment_normalized_tumor*
            reinforce the role of texture-based descriptors.

    <!-- -->

    -   **Visualizations:**

        -   **Boxplots and KDE Plots:** Show moderate separation between
            ER status groups, but significant overlap remains.

        -   **t-SNE and UMAP:** Both reveal overlapping clusters, with
            no clear separation between classes.

<!-- -->

-   **Conclusion:** The KS test effectively identifies features with
    significant distributional differences, but the lack of clear
    separation in visualizations suggests that these features alone are
    insufficient for classification. More complex interactions or
    non-linear relationships may need to be explored.

### <a id='toc1_4_4_'></a>[**4. Non-Linear Dimensionality Reduction + Robust Scaling (Approach 4)**](#toc0_)

-   **Steps:**

    -   Applied Robust Scaling to handle outliers and non-normal
        distributions.

    -   Used UMAP for non-linear dimensionality reduction, achieving a
        trustworthiness score of 0.865.

    -   Identified the top 10 UMAP components using the Point-Biserial
        Correlation Coefficient.

-   **Key Insights:**

    -   **UMAP Performance:** The high trustworthiness score indicates
        that UMAP effectively preserves local and global data
        structures, making it suitable for high-dimensional datasets.

    -   **Feature Importance:** The top 10 UMAP components show weak
        linear correlations with ER status (ranging from 0.0730 to
        0.0441), suggesting that linear methods may not capture the
        underlying relationships effectively.

    -   **Visualizations:**

        -   **UMAP and t-SNE** reveal complex, non-linear structures in
            the data, but no clear separation between ER status groups
            is observed in 2D visualizations.

-   **Conclusion:** While UMAP does not yield striking 2D separation, it
    may still be valuable for advanced non-linear classification
    algorithms (e.g., Random Forests, Gradient Boosting, or Neural
    Networks) that can exploit the preserved local and global
    structures.

### <a id='toc1_4_5_'></a>[**General Observations and Recommendations**](#toc0_)

1.  **Feature Redundancy:**

    -   The high correlation among features (341 out of 529 with r >
        0.8) underscores the need for dimensionality reduction or
        feature selection to improve model performance and
        interpretability.

2.  **Class Separation:**

    -   While some features (e.g., tumor size, texture, and enhancement
        metrics) show moderate separation between ER status groups, the
        classes are not easily separable using the current feature set.

3.  **Outliers:**

    -   Isolation Forest effectively identifies outliers, but their
        impact on model performance should be further investigated.

4.  **Non-Linear Relationships:**

    -   UMAP and t-SNE reveal complex, non-linear structures in the
        data, suggesting that non-linear models may outperform linear
        ones.

5.  **Top-10 Features:**

    -   Both Approaches 2 and 3 consistently identify **tumor size**
        (*Volume_cu_mm_Tumor*) and **morphological metrics**
        (*TumorMajorAxisLength*) as important features.

    -   **Texture and enhancement features** (e.g.,
        *WashinRate_map_Cluster_Prominence_tumor* and
        *Grouping_based_proportion_of_tumor_voxels_3D_tumor_Group_1*)
        also appear in both lists, indicating their discriminative power
        for ER status.

    -   Approach 2 emphasizes *WashinRate_map-based features* related to
        texture and contrast, while Approach 3 highlights *SER-based
        volumetric and mean intensity measurements*. This reflects how
        different feature-selection criteria (correlation thresholds vs.
        statistical distribution tests) can surface complementary sets
        of important radiomic features.

### <a id='toc1_4_6_'></a>[**Summary**](#toc0_)
In my modeling I used 4 approaches:  
1. Approach 1 (Baseline):
To establish a straightforward reference model and assess the impact of subsequent preprocessing or feature engineering steps.  

2. Approach 2 (Correlation-Based Feature Selection & Outlier Removal):
To reduce redundancy in highly correlated MRI features (minimizing multicollinearity and overfitting) and remove anomalous data points that could skew model performance.  

3. Approach 3 (Statistical Distribution–Based Selection via KS Test):
To identify features with significantly different distributions across ER statuses, particularly suitable for non-normal radiomic data and robust to unequal group sizes.  

4. Approach 4 (Non-Linear Dimensionality Reduction + Robust Scaling):
To handle skewed, non-Gaussian features via outlier-resistant scaling and to capture complex, non-linear relationships using UMAP, which balances local and global data structure.

The analysis highlights the importance of tumor size, morphology, and
enhancement texture features in distinguishing ER status. However, the
moderate class separation and weak linear correlations suggest that more
sophisticated modeling approaches are needed to fully exploit the
dataset's potential. By leveraging non-linear algorithms and advanced
feature engineering techniques, it may be possible to achieve better
classification performance and gain deeper insights into the
relationship between MRI-derived radiomic features and ER status.